In [ ]:

from collections import defaultdict
import copy

class CFG:
    def __init__(self, nonterminals, terminals, start, productions):
        self.N = set(nonterminals)
        self.T = set(terminals)
        self.S = start
        self.P = defaultdict(list)
        for A, rhss in productions.items():
            for rhs in rhss:
                self.P[A].append(tuple(rhs))

    def __str__(self):
        lines = []
        for A in sorted(self.P):
            rhss = ["".join(rhs) if rhs else "ε" for rhs in self.P[A]]
            lines.append(f"{A} -> {' | '.join(rhss)}")
        return "\n".join(lines)

def to_cnf(cfg):
    """Convert the given CFG into Chomsky Normal Form (CNF)."""
    new_cfg = copy.deepcopy(cfg)

    if any(new_cfg.S in rhs for rhss in new_cfg.P.values() for rhs in rhss):
        S0 = new_cfg.S + "_S0"
        new_cfg.N.add(S0)
        new_cfg.P[S0].append((new_cfg.S,))
        new_cfg.S = S0

    nullable = set()
    changed = True
    while changed:
        changed = False
        for A, rhss in new_cfg.P.items():
            for rhs in rhss:
                if rhs == () or all(s in nullable for s in rhs):
                    if A not in nullable:
                        nullable.add(A)
                        changed = True

    newP = defaultdict(list)
    for A, rhss in new_cfg.P.items():
        for rhs in rhss:
            if rhs == ():
                continue
            positions = [i for i, s in enumerate(rhs) if s in nullable]
            for i in range(1 << len(positions)):
                to_remove = {positions[j] for j in range(len(positions)) if (i >> j) & 1}
                new_rhs = tuple(rhs[k] for k in range(len(rhs)) if k not in to_remove)
                if new_rhs not in newP[A]:
                    newP[A].append(new_rhs)
    if new_cfg.S in nullable:
        newP[new_cfg.S].append(())
    new_cfg.P = newP

    unit_pairs = set()
    for A in new_cfg.N:
        stack = [A]
        visited = set()
        while stack:
            B = stack.pop()
            for rhs in new_cfg.P.get(B, []):
                if len(rhs) == 1 and rhs[0] in new_cfg.N:
                    if (A, rhs[0]) not in unit_pairs:
                        unit_pairs.add((A, rhs[0]))
                        stack.append(rhs[0])
    for (A, B) in unit_pairs:
        for rhs in new_cfg.P.get(B, []):
            if len(rhs) != 1 or rhs[0] not in new_cfg.N:
                if rhs not in new_cfg.P[A]:
                    new_cfg.P[A].append(rhs)

    term_map = {}
    for A in list(new_cfg.P.keys()):
        for rhs in list(new_cfg.P[A]):
            if len(rhs) > 1:
                new_rhs = []
                for s in rhs:
                    if s in new_cfg.T:
                        if s not in term_map:
                            X = f"T_{s}"
                            term_map[s] = X
                            new_cfg.N.add(X)
                            new_cfg.P[X].append((s,))
                        new_rhs.append(term_map[s])
                    else:
                        new_rhs.append(s)
                new_cfg.P[A].remove(rhs)
                new_cfg.P[A].append(tuple(new_rhs))

    counter = 1
    for A in list(new_cfg.P.keys()):
        for rhs in list(new_cfg.P[A]):
            if len(rhs) > 2:
                new_cfg.P[A].remove(rhs)
                prev = rhs[0]
                for i in range(1, len(rhs) - 1):
                    Y = f"X{counter}"
                    counter += 1
                    new_cfg.N.add(Y)
                    new_cfg.P[Y].append((rhs[i], rhs[i + 1] if i + 1 == len(rhs) - 1 else f"X{counter}"))
                    new_cfg.P[A].append((prev, Y))
                    break

    return new_cfg

nonterminals = {'S', 'A', 'B'}
terminals = {'a', 'b'}
start = 'S'
productions = {
    'S': [('A', 'S', 'A'), ('a', 'B')],
    'A': [('B',), ('S',)],
    'B': [('b',), ()]
}

cfg = CFG(nonterminals, terminals, start, productions)
print("Original Grammar:")
print(cfg, "\n")

cnf = to_cnf(cfg)
print("Grammar in Chomsky Normal Form:")
print(cnf)


Original Grammar:
A -> B | S
B -> b | ε
S -> ASA | aB 

Grammar in Chomsky Normal Form:
A -> B | ε | S | b | a | SA | AS | T_aB | AX2
B -> b
S -> S | a | SA | AS | T_aB | AX1
S_S0 -> S | a | SA | AS | T_aB | AX3
T_a -> a
X1 -> SA
X2 -> SA
X3 -> SA
